In [1]:
# importing tools 
import s3fs
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt 
import math

In [2]:
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })

In [3]:
fsg.ls('data')

['data/CREG12.L75-REF08_mesh_zgr.zarr',
 'data/bathym.nc',
 'data/botvel.nc',
 'data/u_names.nc',
 'data/u_tot.nc',
 'data/ubar.nc',
 'data/uvar.nc',
 'data/uvar_vvar',
 'data/v_names.nc',
 'data/v_tot.nc',
 'data/vbar.nc']

In [4]:
data_path = 's3://data/*.nc'#, 's3://data/vvar.nc', 's3://data/uv.nc', 's3://data/bathym.nc', 's3://data/.nc'


In [5]:
remote_files = fsg.glob(data_path)
remote_files

['data/bathym.nc',
 'data/botvel.nc',
 'data/u_names.nc',
 'data/u_tot.nc',
 'data/ubar.nc',
 'data/uvar.nc',
 'data/v_names.nc',
 'data/v_tot.nc',
 'data/vbar.nc']

In [8]:
fileset = [fsg.open(file) for file in remote_files]

In [10]:
# This works
dset = xr.open_mfdataset(fileset, combine='by_coords')

ValueError: cannot guess the engine, file-like object read/write pointer not at the start of the file, please close and reopen, or use a context manager

In [13]:
uvar = xr.open_mfdataset(fileset)
#vvar = xr.open_mfdataset(filedir + 'vvar.nc')
#uv = xr.open_mfdataset(filedir + 'uv.nc')
#uv = uv.rename({'__xarray_dataarray_variable__' : 'vel'})
#bat = xr.open_mfdataset(filedir + '/CREG12.L75.script/bathym.nc')
#dyp = xr.open_mfdataset(filedir + 'depth.nc')


In [14]:
uvar

<xarray.Dataset>
Dimensions:        (depth: 75, x: 1580, y: 1801)
Coordinates:
    nav_lat        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    time_centered  object ...
    time_counter   object ...
Dimensions without coordinates: x, y
Data variables:
    vozocrtx       (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>

In [ ]:
# Smoothen data, over every tenth point in both x and y direction, for all the data
uvar = uvar.rolling(x=10).mean()
vvar = vvar.rolling(y=10).mean()
uv = uv.rolling(x=10).mean()
uv = uv.rolling(y=10).mean()

In [ ]:
# List of x,y elements that I wanna plot over later 
dx = 10
dy = 10

In [ ]:
# what is this for again? jmax,imax?
jmax, imax = uv.vel[0].shape
i_list = np.arange(200,1400,dx)
j_list = np.arange(800,1800,dy)

In [ ]:
# Making the covmatrix and finding the eigenvalues and eigenvectors
# list of all the points for the PC-crosses
PC11 = np.zeros(3379)
PC12 = np.zeros(3379)
PC21 = np.zeros(3379)
PC22 = np.zeros(3379)
y = np.zeros(3379)
x = np.zeros(3379)
indx = 0


In [ ]:
# to get the loop to run faster
uvy = np.array(uv.y.values)[800:1800]
uvx = np.array(uv.x.values)[200:1400]


In [ ]:
# chooseing a level to look at 
level = 7
depth = dyp.depth[level].values
print(depth)

In [ ]:
# Level values
uvar0 = np.array(uvar.vozocrtx[level,800:1800,200:1400].values)
vvar0 = np.array(vvar.vomecrty[level,800:1800,200:1400].values)
uvvel = np.array(uv.vel[level,800:1800,200:1400].values)

In [ ]:
print(uvar0.shape)
print(uvvel.shape)

In [ ]:
for i in i_list:
    for j in j_list:
        covm = np.array([[uvar0[j,i],uvvel[j,i]],
        [uvvel[j,i],vvar0[j,i]]],dtype=float)
        if np.isnan(covm).any():
            continue
        else:
            values, vectors = np.linalg.eig(covm)
            PC11[int(indx)] = vectors[0,0]*np.sqrt(values[0])
            PC12[int(indx)] = vectors[1,0]*np.sqrt(values[0])
            PC21[int(indx)] = vectors[0,1]*np.sqrt(values[1])
            PC22[int(indx)] = vectors[1,1]*np.sqrt(values[1])
            y[int(indx)] = uvy[j]
            x[int(indx)] = uvx[i]
            indx = indx + 1

In [ ]:
# converting to np.arrays
PC11 = np.array(PC11,dtype=float)
PC12 = np.array(PC12,dtype=float)
PC21 = np.array(PC21,dtype=float)
PC22 = np.array(PC22,dtype=float)
y = np.array(y,dtype=float)
x = np.array(x,dtype=float)

In [ ]:
# Plot
fig, ax = plt.subplots(1,1,figsize=(10,10))
cmap = plt.get_cmap('magma_r')
ax.contour(bat.mbathy, colors='grey',levels=8, linewidths=0.4)
ax.quiver( x[::2], y[::2], PC11[::2], PC12[::2], 
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=5)
ax.quiver(x[::2], y[::2], -PC11[::2], -PC12[::2],
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=5)
ax.quiver( x[::2], y[::2], PC21[::2], PC22[::2],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=5)
ax.quiver( x[::2], y[::2], -PC21[::2], -PC22[::2],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=5)
ax.set_xlim(200,1400)
ax.set_ylim(800,1800)

plt.title('PC corsses plottet at ', depth ,'m')
plt.show()